# ***scalar.py***

Il programma è strutturato principalmente in due classi:

* *ScalarModel*
* *TabularLearner*

## *ScalarModel*

* **Inizializzazione**: 
    ``` python
    def __init__(self, reward, draw=False)
    ```
    Questa classe modella l'ambiente cellulare e gestisce il comportamento delle cellule nel tempo. Il costruttore inizializza il modello con il tipo di ricompensa (*reward*) e se disegnare o meno il grafico dell'evoluzione delle cellule (*draw*).

* **Reset**: 
    ``` python
    def reset(self)
    ```
    La funzione *reset* reimposta lo stato dell'ambiente cellulare, inizializzando il tempo, la quantità di glucosio e ossigeno, e crea una popolazione di cellule sane e cancerose. Con la riga seguente crea una lista contenente una cellula cancerosa e 1000 cellule sane. Tutti gli oggetti "cellula", sono creati passando il parametro 0. PERCHE? COME MAI CREO SOLO UNA CELLULA CANCEROSA?
    ``` python
    self.cells = [CancerCell(0)] + [HealthyCell(0) for _ in range(1000)]
    ```

* **Ciclo delle cellule**:
    ``` python
    def cycle_cells(self):
        ...
    ```
    Questa funzione simula il ciclo di vita delle cellule, dove ogni cellula consuma risorse e può dividersi (mitosi) se le condizioni lo permettono.

    *   ``` python
        to_add = []
        ```
        La lista *to_add* viene utilizzata per raccogliere le nuove cellule che si formeranno durante il ciclo.

    *   ``` python
        for cell in random.sample(self.cells, count):
        ```
        La funzione itera su un campione casuale di cellule. *random.sample(self.cells, count)* restituisce un campione di count cellule dall'elenco delle cellule attuali.


    *   ``` python
        res = cell.cycle(self.glucose, count / 278 , self.oxygen)
        ```
        Ogni cellula esegue il proprio ciclo di vita chiamando il metodo *cycle* della cellula stessa. Questo metodo riceve come parametri la quantità di glucosio disponibile, una frazione del numero totale di cellule (*count / 278*), e la quantità di ossigeno disponibile. *res* è una lista che contiene: Il consumo di glucosio della cellula, Il consumo di glucosio della cellula e un terzo valore opzionale che indica se la cellula si è divisa (mitosi) e il tipo di cellula risultante (sana o cancerosa). *count / 278* sarebbe il numero di cellule vicine ma siccome in questo caso siamo in una dimensione assumiamo quel numero (ç).

    *   ``` python
        if len(res) > 2:
            if res[2] == 0:
                to_add.append(HealthyCell(0))
            elif res[2] == 1:
                to_add.append(CancerCell(0))
        ```
        Se *res* contiene più di due elementi, significa che la cellula si è divisa. Se *res[2]* è 0, una nuova cellula sana viene aggiunta alla lista to_add. Se *res[2]* è 1, una nuova cellula cancerosa viene aggiunta alla lista to_add.

    *   ``` python
        self.glucose -= res[0]
        self.oxygen -= res[1]
        ```

        La quantità di glucosio e ossigeno nell'ambiente viene ridotta in base al consumo delle cellule (*res[0]* e *res[1]* rispettivamente).
    
    *   ``` python
        self.cells = [cell for cell in self.cells if cell.alive] + to_add
        ```
        L'elenco delle cellule viene aggiornato per includere solo le cellule che sono ancora vive (*cell.alive*) e per aggiungere le nuove cellule formate durante il ciclo (*to_add*).

* **Riempimento delle risorse**:
    ``` python
    def fill_sources(self):
    ```
    La funzione fill_sources() aggiunge una quantità fissa di glucosio (13000) e ossigeno (450000) all'ambiente. Questo simula l'apporto costante di risorse necessarie per la sopravvivenza delle cellule.


* **Avanzamento del tempo**: 
    ``` python
    def go(self, ticks=1):
    ```
    La funzione è responsabile dell'avanzamento del tempo nell'ambiente cellulare e dell'aggiornamento dello stato delle risorse e delle cellule.

    *   *ticks*: Un intero che rappresenta il numero di unità di tempo (ore) per cui avanzare la simulazione. Il valore predefinito è 1

    * self.time += 1
    Ogni iterazione del ciclo incrementa la variabile time di 1, rappresentando un'ora che passa nella simulazione.
    
    *   ``` python
        self.cycle_cells()
        ```
        La funzione cycle_cells() aggiorna lo stato di ogni cellula nell'ambiente. Ogni cellula consuma risorse (glucosio e ossigeno) e può dividersi (mitosi) se le condizioni lo permettono. Le cellule morte vengono rimosse dalla lista self.cells e le nuove cellule vengono aggiunte.

    *   ``` python
        if self.draw_graph:
        ```
        Se l'attributo draw_graph è True, vengono aggiornati i dati per il grafico. I seguenti dati vengono aggiunti alle rispettive liste: *self.ticks*: La lista dei tempi (ore), *self.ccell_counts*: La lista dei conteggi delle cellule cancerose e  *self.hcell_counts*: La lista dei conteggi delle cellule sane.

* **Irradiazione**:
    ``` python
    def irradiate(self, dose):
    ```
    Applica una dose di radiazione a tutte le cellule, determinando la loro sopravvivenza o morte in base alla dose ricevuta.

* **Osservazione**:
    ``` python
    def observe(self):
        return HealthyCell.cell_count, CancerCell.cell_count
    ```
    Ritorna il conteggio delle cellule sane e cancerose.

* **Azione**:
    La funzione act applica una dose di radiazione (determinata dall'azione) e avanza il tempo di 24 unità, ritornando la ricompensa aggiustata in base alla dose e agli effetti sulle cellule. 24 unità corrispondono a 24 ore (ç)
    * Calcolo e Ritorno della Ricompensa
        ``` python
        return self.adjust_reward(dose, pre_ccell - post_ccell, pre_hcell - min(post_hcell, m_hcell))
        ```
        Vedi punto seguente

* **Aggiustamento della ricompensa**
    ``` python
    def adjust_reward(self, dose, ccell_killed, hcells_lost):
    ```
    Calcola la ricompensa in base alla dose somministrata, al numero di cellule cancerose uccise e al numero di cellule sane perse. Di seguito sono riportate le formule per le ricompense:

    * Se lo stato è terminale:
        * *Se self.end_type* è *'L'* o *'T'*:
        $$R = -1$$

        * Altrimenti:
            * Se *self.reward* è *'dose'*:
            $$R = -\frac{\text{dose}}{200} + 0.5 + \frac{\text{HealthyCell.cell\_count}}{4000}$$

            * Altrimenti:
            $$R = 0.5 + \frac{\text{HealthyCell.cell\_count}}{4000}$$

    * Se lo stato non è terminale: 
        * Se *self.reward* è *'dose'* oppure *'oar'*:
        $$R = -\frac{\text{dose}}{200} + \frac{\text{ccell\_killed} - 5.0 \times \text{hcells\_lost}}{100000}$$

        * Se *self.reward* è *'killed'*:
        $$R = \frac{\text{ccell\_killed} - \text{reward} \times \text{hcells\_lost}}{100000}$$


    Il fattore 5.0 indica che la perdita di cellule sane ha un peso negativo cinque volte maggiore rispetto all'uccisione delle cellule cancerose.
    
    La funzione *'killed'* corrisponde all’obiettivo di massimizzare il danno alle cellule cancerose riducendo al minimo il danno alle cellule sane,*'dose'* aggiunge un parametro, portando l’agente a trovare trattamenti con dosi totali più piccole, poiché gli effetti a lungo termine correlati a questa dose totale non sono modellati nella simulazione.

    Nota: quando leggi il paper, Killed and dose fa riferimento a *'dose'* nel codice mentre Kelled corrisponde a *'killed'*. (§ Controlla formule reward nel codice e nel paper. C'è una differenza nelle formule che può essere dovuta al fatto che questo codice corrisponde al caso scalare)

    
* **Stato terminale**: 
    ``` python
    def inTerminalState(self):
    ```
    Determina se l'ambiente è in uno stato terminale (vittoria, sconfitta, o tempo esaurito).

* **Disegno del grafico**: 
    ``` python
    def draw(self, title):
    ```
    Disegna un grafico dell'evoluzione delle cellule nel tempo.



## *TabularLearner*

Questa classe implementa l'algoritmo di Q-learning tabulare per ottimizzare il trattamento radioterapico.

